<a href="https://colab.research.google.com/github/AriYusa/meeting-info-extraction/blob/main/Meeting%20extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Извлечение информации о встрече

### Импорт необходимых библиотек

In [409]:
import json
import logging
import sys
import csv
import re
import random

import pandas as pd
import numpy as np

import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans
from spacy import displacy


# Подготовка данных

In [410]:
# предложения считываются и записываюся в список
train_dataset=[]
with open("raw_dataset.csv", "r") as f:
    reader = csv.reader(f, delimiter=';')
    headers = next(reader, None) 
    for row in reader:
      train_dataset.append(row[0])

In [411]:
# предложения разбиваются на токены и записываются в формате, удобном для разметки
dataset=[]
for i,text in enumerate(train_dataset):
  words = text.split()
  for word in words:
    dataset.append([i, word])

with open("output2.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["Sentence#", "word"])
    writer.writerows(dataset)

In [412]:
# Конвертируем размеченный даиасет в формат, подходящий для spacy
def csv_to_json_format(input_path):
        f=open(input_path,'r') # input file
        reader = csv.reader(f, delimiter=';')
        headers = next(reader, None)
        #fp=open(output_path, 'w') # output file
        
        data_dict={}
        train_data = []
        prev_sent_i = 0
        current = 0
        sentence = ''
        entities=[]
        prev_label = 'O'
        label_end, label_start = 0, 0

        for line in reader:

          sent_i, word, label = line
          sent_i = int(sent_i)
          if sent_i != prev_sent_i:

            if (prev_label != 'O'):
              label_end = current-2
              entities.append((label_start, label_end, prev_label))

            train_data.append((sentence[:-1],entities))

            current = 0
            sentence = ''
            entities=[]
            prev_label = 'O'
            label_end, label_start = 0, 0

          if (label != prev_label):
            if (prev_label != 'O'):
              label_end = current-1
              entities.append((label_start, label_end, prev_label))
            label_start = current
              
          sentence += word + ' '
          current += len(word) + 1
          prev_label = label
          prev_sent_i = sent_i


        if (prev_label != 'O'):
          label_end = current-2
          entities.append((label_start, label_end, prev_label))

        train_data.append((sentence[:-1],entities))

        return train_data

In [413]:
nlp = spacy.load('en_core_web_sm')
training_data = csv_to_json_format("/content/labeled_dataset.csv")

In [414]:
ents=[]

# the DocBin will store the example documents
db = DocBin()
for text, annotations in training_data:
    doc = nlp(text)
    doc.spans["sc"] = []
    ents=[]
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label, alignment_mode = "contract")
        doc.spans["sc"].append(span)
        ents.append(span)
    filtered = filter_spans(ents)
    doc.ents = filtered
    db.add(doc)
db.to_disk("./train.spacy")

In [415]:
# теперь можно увидеть разметку
displacy.render(doc, style="span", jupyter=True)

## Вариант 1. Обучение NER

In [416]:
# инициализируем конфирурацию обучаемой модели
! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [417]:
# обучаем модель
! python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-07-14 21:52:13,853] [INFO] Set up nlp object from config
[2022-07-14 21:52:13,865] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-07-14 21:52:13,871] [INFO] Created vocabulary
[2022-07-14 21:52:13,872] [INFO] Finished initializing nlp object
[2022-07-14 21:52:14,184] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     47.20    0.00    0.00    0.00    0.00
 13     200        212.80   2303.49   97.51   98.00   97.03    0.98
 29     400         22.51     38.53  100.00  100.00  100.00    1.00
 49     

In [418]:
test_dataset = [
                "What about dinner at the Ginny's at 7pm?",
                "Waiting for you in the school tomorrow!",
                "I have scheduled a meeting of the auditors for 1:00-5:00 p.m., June 6, in conference room.",
                "I’ll meet you by the main reception desk at 8.00",
                "Why don’t we meet for lunch on Friday?",
                "We arranged to meet outside the theatre.",
                "We met when we were at college.",
                "The two leaders are scheduled to meet again next month in a town near the border.",
                "Fans began to gather outside the stadium three hours before the start of the match."
]

In [419]:
nlp1 = spacy.load(R"/content/output/model-best") #load the best model

In [420]:
# смотрим не результаты на тестовом датасете
for text in test_dataset:
  doc = nlp1(text) 
  spacy.displacy.render(doc, style="ent", jupyter=True)

/usr/local/lib/python3.7/dist-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


Видно, что модель показала низкое качество: где-то выделила сущности частично, где-то совсем не распознала.

Для высокого качества обученой необходимо намного больше данных.

## Вариант 2. Rule-based

In [435]:
classic_nlp = spacy.load('en_core_web_sm')
doc = classic_nlp(text) 

Посмотрим на встренную NER-модель в spacy. Видно, что и она тоже не всегда точно распознаёт сущности: например, `the stadium three hours` выделено как  `TIME`

In [436]:
for text in test_dataset:
  doc = classic_nlp(text) 
  spacy.displacy.render(doc, style="ent", jupyter=True)

/usr/local/lib/python3.7/dist-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


Spacy умеет классифицировать и визуализировать зависимости между словами.

In [437]:
displacy.render(doc, style='dep', jupyter=True)

Посмотрим, какие свойсва есть у токенов: с помощью заранее обученных моделей Spacy пожет предсказывать часть речи, роль в предложении, зависимые слова и т.д.

In [438]:
parse_tree = []

for token in doc:
    parse_tree.append([token.i, token.text, token.dep_, token.ent_type_, token.head.i, token.head.text, token.head.pos_, [child.i for child in token.children]])
    
df = pd.DataFrame(parse_tree, columns=['index','text','dep', 'ent', 'parent_index','parent','parent_pos', 'childrens_index'])

In [439]:
df

,index,text,dep,ent,parent_index,parent,parent_pos,childrens_index
0,0,Fans,nsubj,,1,began,VERB,[]
1,1,began,ROOT,,1,began,VERB,"[0, 3, 15]"
2,2,to,aux,,3,gather,VERB,[]
3,3,gather,xcomp,,1,began,VERB,"[2, 4, 9]"
4,4,outside,prep,,3,gather,VERB,[6]
5,5,the,det,TIME,6,stadium,NOUN,[]
6,6,stadium,pobj,TIME,4,outside,ADP,[5]
7,7,three,nummod,TIME,8,hours,NOUN,[]
8,8,hours,npadvmod,TIME,9,before,ADP,[7]
9,9,before,prep,,3,gather,VERB,"[8, 11]"


Создадим класс для поиска информации о встрече на основе данных, которые можно получить из свойств токена в Spacy.

In [445]:
class EventExtractor:
  def __init__(self, doc):
    self.doc = doc
    self.roots = [token for token in self.doc if token.head == token] # roots - корневые токены, те в которые не входит ни одной зависимости, только исходят

    self.features = []
    for i in range(len(self.roots)):
      self.features.append({'TIME':[],
                      'LOC': []})
  
  def extract(self):
    """
    Метод для извлечения информации
    """
    for r_i, root in enumerate(self.roots):
      self.root_i = r_i
      self.check_childrens(root, go_deeper=True)


  def check_childrens(self, token, go_deeper=False):
    """
    Метод для итеративной проверки дочерних токенов.
    """
    for child in token.children:
      if child.dep_ in ['prep', 'npadvmod', 'advmod']:
        self.check_descendants(child)
      if go_deeper and (self.features[self.root_i]['TIME'] is not None and self.features[self.root_i]['LOC'] is not None):
        self.check_childrens(child)


  def check_descendants(self, token):
    """
    Метод для итеративной провери всех предков, включая сам токен
    """
    for child in token.subtree:
      if self.search_for_loc(child, token):
        break
    for child in token.subtree:
      if self.search_for_time(child, token):
        break

            
  def search_for_time(self, token, ancestor):
    """
    Извлечение информации о времени.
    """
    # если условие выполняется, извлекается не только токен, но и все дочерние сущности ближайшего предка 
    if token.ent_type_ in ['TIME', 'DATE'] or token.dep_ in ['npadvmod', 'nummod', 'nummod'] or token.pos_ == 'NUM' or token.text in ['pm', 'am']:
      subtree = [t.text for t in ancestor.subtree]
      if self.features[self.root_i]['TIME'][-len(subtree):] != subtree: # чтобы информация не дулировалась
        self.features[self.root_i]['TIME'].extend(subtree)
        return True
    return False

  def search_for_loc(self, token, ancestor):
    """
    Извлечение информации о месте.
    """ 
    if token.ent_type_ in ['LOC', 'GPE', 'FAC']:
        self.features[self.root_i]['LOC'].extend([t.text for t in ancestor.subtree]) 
        return True
    childrens_index = [child.i for child in token.children]
    if childrens_index != []:
      if ancestor.pos_ == 'ADP' and ancestor.text not in ['as', 'for'] and doc[childrens_index[0]].pos_ == 'DET' and not np.any([child.ent_type_ in ['TIME', 'DATE'] for child in ancestor.subtree]):
          self.features[self.root_i]['LOC'].extend([t.text for t in ancestor.subtree])
          return True
    return False  

  
  def display_info(self):
    """
    Вывод информации в удобном формате
    """
    print("================================================================")
    print("SENTENCE:", self.doc.text)
    for info_set in self.features:
      print("----------------------------------------------------------------")
      if info_set['TIME'] == [] and info_set['LOC'] == []:
        print("This sentence does't have a meeting info.")
        continue
      print("MEETING EVENT")
      time = (" ").join(info_set['TIME']).capitalize()
      loc = (" ").join(info_set['LOC']).capitalize()
      print("WHEN:", time if time!="" else "Unknown")
      print("WHERE:", loc if loc!="" else "Unknown")
    print(" ")

Проверим модель на случайной выборке из датасета. Для этой модели не нужны обучающие данные (при условии что модели Spacy (PoS tagger, NER  и др.) можно использовать обученными по умолчанию). Это одно из преимуществ Rule-Based моделей

In [441]:
full_dataset = train_dataset+test_dataset

random.seed(2)
sample_dataset = random.sample(full_dataset, 10)

In [447]:
for text in sample_dataset:
  # displacy.render(doc, style='dep', jupyter=True)
  # displacy.render(doc, style='ent', jupyter=True)
  text = re.sub(r'\xa0', ' ', text)
  doc = classic_nlp(text)
  extr_obj = EventExtractor(doc)
  extr_obj.extract()
  extr_obj.display_info()

SENTENCE: I am very glad to announce that next Friday the 27th of March we will have special guests with us who will present their best practices with us.
----------------------------------------------------------------
This sentence does't have a meeting info.
 
SENTENCE: In ice hockey, the Colorado Avalanche defeat the Tampa Bay Lightning to win the Stanley Cup (Conn Smythe Trophy winner Cale Makar pictured)
----------------------------------------------------------------
This sentence does't have a meeting info.
 
SENTENCE: Mary’s attorney issues a subpoena ad testificandum to one of Mary’s co-workers, who she says witnessed her supervisor’s sexual advances on more than one occasion. The co-worker is required to attend a deposition on May 25th, at 9:00 a.m., at the office of her attorney to give testimony.
----------------------------------------------------------------
MEETING EVENT
WHEN: To one of mary ’s co - workers ,
WHERE: Unknown
----------------------------------------------

Эта модель справилась с задачей гораздо лучше, чем передудущая:
- В некоторых предложениях они отпределила абсолютно точно и время, и локацию. Например, `WHEN: In 1970`; `WHERE: In new york city`
- В некоторых предложениях нашла все компоненты времени: дата, время, и даже продолжительность и часовой пояс! Например, `WHEN: On saturday , november 13 from 10:00 to 20:00 moscow time`
- В некоторых предложениях нашла даже сложные конструкции. Например, `WHERE: At the cemetery of santa ifigenia`

Однако модель иногда все же допускает ошибки:
- Путает дату и локацию
- Извлекает сущьности лишь  частично

Большинсво этих ошибок связано со сложной структурой предложений. Так как наша модель во многом опирается на результаты модели Spacy, предсказыающей связи между словами, то в случае некорректного предсказания можно получить неожидааемый результат.

Дальнейшее уточнение модели `EventExtractor` и улучшение качества моделей Spacy, гарантированно приведёт к более точному извлечению информации в будущем.